# 2. Imports

## 2.1. Libraries

In [27]:
import math
import pandas as pd
import inflection

## 2.2. Helper Functions

In [13]:
def change_cols_to_snake_case(df):
    '''
    Change a list of columns of dataframe from 
    camel case to snake case.
    This function returns the dataframe modified.
    '''
    cols = df.columns
    snake_case = lambda x: inflection.underscore(x)
    new_cols = list(map(snake_case, cols))
    df.columns = new_cols
    return df

## 2.3. Loading Data

In [8]:
# Loading the sales df
df_sales = pd.read_csv('data/train.csv', low_memory=False)
display(df_sales.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [9]:
# Loading the store df
df_store = pd.read_csv('data/store.csv', low_memory=False)
display(df_store.head())

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [10]:
# Merging the sale and store df's
df = pd.merge(df_sales, df_store, how='left', on='Store')
display(df.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# 3. Data Description

Most of the fields are self-explanatory. The following are descriptions for those that aren't.

- `Id` - an Id that represents a (Store, Date) duple within the test set;

- `Store` - a unique Id for each store;

- `Sales` - the turnover for any given day (this is what you are predicting);

- `Customers` - the number of customers on a given day;

- `Open` - an indicator for whether the store was open: 
    - 0 = closed; 
    - 1 = open.

- `StateHoliday` - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. 
    - a = public holiday;
    - b = Easter holiday;
    - c = Christmas;
    - 0 = None.

- `SchoolHoliday` - indicates if the (Store, Date) was affected by the closure of public schools;

- `StoreType` - differentiates between 4 different store models: a, b, c, d;

- `Assortment` - describes an assortment level: 
    - a = basic;
    - b = extra;
    - c = extended.

- `CompetitionDistance` - distance in meters to the nearest competitor store;

- `CompetitionOpenSince[Month/Year]` - gives the approximate year and month of the time the nearest competitor was opened;

- `Promo` - indicates whether a store is running a promo on that day;

- `Promo2` - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating;

- `Promo2Since[Year/Week]` - describes the year and calendar week when the store started participating in Promo2;

- `PromoInterval` - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.

## 3.1. Rename Columns

In [17]:
# Showing the old columns
print('Old columns:')
print(df.columns, '\n')

# Changing columns to snake case
print('New columns:')
df = change_cols_to_snake_case(df)
print(df.columns)

Old columns:
Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object') 

New columns:
Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')


## 3.2. Data Dimensions

In [20]:
print(f'Number of rows: {df.shape[0]};')
print(f'Number of columns: {df.shape[1]}.')

Number of rows: 1017209;
Number of columns: 18.


## 3.3. Data Types

In [23]:
print(df.dtypes)

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object


In [24]:
# Changing the date type to datetime
df['date'] = pd.to_datetime(df['date'])
print(df.dtypes)

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object


## 3.4. Check NaN

In [26]:
print(df.isna().sum())

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64


## 3.5. Fillout NaN

In [ ]:
print(f'The ')

In [ ]:
# competition_distance
df['competition_distance'] = df['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

## 3.6.

## 3.7.

## 3.8.